In [33]:
library(conflicted)
library(plm)
library(psych)
library(readxl)
library(splm)
library(tidyverse)
conflict_prefer("lag", "stats")
conflict_prefer("filter", "dplyr")

adj_mtx <- read_csv(
    file.path(path.expand("~"), "OneDrive", "Rawdata", "Geographic Information Data", "adj_mtx.csv"),
) %>% column_to_rownames(., var = "GID_0")
countrycode <- read_excel(
    file.path(path.expand("~"), "OneDrive", "Rawdata", "Country Code", "Countrycode.xlsx"),
)
df <- pdata.frame(read_excel(file.path(path.expand("~"), "OneDrive", "PhD Dissertation", "Data_Code", "Data", "index_data_regvariables.xlsx")),
    index = c("Numeric", "Year")
)
df["lnFDIstock_w"] <- as.vector(winsor(df["lnFDIstock"], trim = .01))
df["lnFDIstock_w2"] <- as.vector(winsor(df["lnFDIstock"], trim = .05))
df["s_lnFDIstock_w"] <- as.vector(scale(df["lnFDIstock_w"]))
df["SDI"] <- df["SDI"] * 100
# df['s_lnGNI'] <- as.vector(scale(df['lnGNI']))
df_sum <- df %>% select("SDI", "lnFDIstock_w", "financial", "urban", "tax", "war_intensity")
df_withoutsdi <- df %>% select("lnFDIstock_w", "financial", "urban", "tax", "war_intensity")


[conflicted] Removing existing preference.
[conflicted] Will prefer stats::lag over any other package.
[conflicted] Removing existing preference.
[conflicted] Will prefer dplyr::filter over any other package.
Rows: 263 Columns: 264
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): GID_0
dbl (263): ABW, AFG, AGO, AIA, ALA, ALB, AND, ARE, ARG, ARM, ASM, ATA, ATF, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`


,AGO,BEN,BWA,BFA,CMR,COD,COG,GMB,GHA,KEN,⋯,MUS,NAM,NER,NGA,RWA,SEN,ZAF,UGA,ZMB,ZWE
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AGO,0,0,0,0,0,1,1,0,0,0,⋯,0,1,0,0,0,0,0,0,1,0
BEN,0,0,0,1,0,0,0,0,0,0,⋯,0,0,1,1,0,0,0,0,0,0
BWA,0,0,0,0,0,0,0,0,0,0,⋯,0,1,0,0,0,0,1,0,0,1
BFA,0,1,0,0,0,0,0,0,1,0,⋯,0,0,1,0,0,0,0,0,0,0
CMR,0,0,0,0,0,0,1,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
COD,1,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,1,0,0,1,1,0
COG,1,0,0,0,1,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GMB,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
GHA,0,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [41]:
roi <- (countrycode %>% filter(Numeric %in% (df$Numeric %>% unique())))[["Alpha-3 code"]]
roi_adj_mtx <- adj_mtx[roi, roi]
roi_adj_mtx["KEN", "MDG"] <- 1
roi_adj_mtx["MDG", "KEN"] <- 1
roi_adj_mtx["MUS", "MDG"] <- 1
roi_adj_mtx["MDG", "MUS"] <- 1
roi_adj_mtx <- roi_adj_mtx %>% as.matrix()

In [49]:
fm <- SDI ~ lnFDIstock + financial + urban + tax + war_intensity + lngnic + WGI
bsktest(fm,data=df, listw = spdep::mat2listw(roi_adj_mtx), test="LM1")

Warning message in sqrt(Vd1):
“NaNs produced”



	Baltagi, Song and Koh SLM1 marginal test

data:  SDI ~ lnFDIstock + financial + urban + tax + war_intensity +     lngnic + WGI
LM1 = NaN, p-value = NA
alternative hypothesis: Random effects


In [51]:
spml(fm, data = df, listw = spdep::mat2listw(roi_adj_mtx), model="within") %>% summary()

Warning message in spdep::mat2listw(roi_adj_mtx):
“style is M (missing); style should be set to a valid value”


Spatial panel fixed effects error model
 

Call:
spml(formula = fm, data = df, listw = spdep::mat2listw(roi_adj_mtx), 
    model = "within")

Residuals:
    Min.  1st Qu.   Median  3rd Qu.     Max. 
-4.34245 -0.76757 -0.09911  0.59802  7.28667 

Spatial error parameter:
    Estimate Std. Error t-value  Pr(>|t|)    
rho 0.104358   0.017654  5.9112 3.397e-09 ***

Coefficients:
               Estimate Std. Error t-value  Pr(>|t|)    
lnFDIstock     0.463410   0.137791  3.3631 0.0007707 ***
financial      0.226341   0.051243  4.4170 1.001e-05 ***
urban         -0.738895   0.199172 -3.7098 0.0002074 ***
tax           -5.562654   4.917438 -1.1312 0.2579668    
war_intensity  0.020025   0.121895  0.1643 0.8695130    
lngnic         1.640657   0.354186  4.6322 3.618e-06 ***
WGI           -0.522717   0.554435 -0.9428 0.3457869    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
